<a href="https://colab.research.google.com/github/cchen744/uhi-extreme-heat-response/blob/main/notebooks/03_uhi_n_landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ΔUHI & Built-environment
1. **Goal**: To investigate the possible relationship between SUHI response and built environment. Since we observed a significant increase in SUHI in several cities while others not, we are assuming that if this condition-dependence might be contributed to by composition of the built environment; we test whether this is true.

2. **Outcome Variable: Defining SUHI condition-dependence**: in this step, we define SUHI's condition-dependence as:
    
    *ΔUHI = UHI_extreme − UHI_baseline*

    - UHI_extreme: SUHI when the daily mean temperature is over 90 percentile of the daily mean temperature
    - UHI_baseline: the average of SUHI when the daily mean temperature is between 50-70 percentile.

    (Percentiles are computed within the warm-season window to ensure comparability across cities.)

3. **Explanatory Variables: Built-environment characteristics**:

    **Surface composition**:
      - Impervious surface fraction
      - Vegetation / NDVI / tree cover
      - Water or bare land fraction
      - LCZ composition

    **Urban form & intensity proxies**:
      - Built-up density / road density

4. **Analytical Strategy**
  - Analytical Unit: grid cell within each city (resolution =
  - Model:
  
    *ΔUHI_cell ~ composition_cell + proxies_cell + city fixed effects*

  - Comparison logic:
    - within-city: which built factors is correlated with ΔUHI_cell
    - across-city: does this explain why some city has higher ΔUHI
  
  - Control principles:
    - Same buffer scale
    - Same spatial resolution
    - Same seasonal window



In [1]:
!git init
!git remote add origin https://github.com/cchen744/uhi-extreme-heat-response.git
!git pull origin main --allow-unrelated-histories
!git branch -m master
!git status

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 272 (delta 58), reused 41 (delta 15), pack-reused 159 (from 2)
Receiving objects: 100% (272/272), 10.56 MiB | 8.82 MiB/s, done.
Resolving deltas: 100% (119/119), done.
From https://github.com/cchen744/uhi-extreme-heat-response
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origi

In [2]:
from pathlib import Path
import os
import pandas as pd
import ee
import uhi_pipeline
import importlib
importlib.reload(uhi_pipeline)
print("uhi_pipeline module reloaded.")

ee.Authenticate()
ee.Initialize(project='extremeweatheruhi')

DATA_DIR = Path("data/city_cell")
DATA_DIR.mkdir(parents=True, exist_ok=True)

ua_fc = ee.FeatureCollection("projects/extremeweatheruhi/assets/uac20_2025")

uhi_pipeline module reloaded.


Before starting analysis, we need to get daily output on the level of grid cell due to the need for intra-city built environment analysis. In uhi_pipeline.py, the grid cell is defined as a 1km x 1km square grid projected from EPSG: 3875.

In [3]:
city_fc=uhi_pipeline.select_ua(ua_fc,ua_contains="Phoenix")
city_geom = city_fc.geometry()
ic = (ee.ImageCollection("MODIS/061/MYD11A1")
      .filterBounds(city_geom)
      .filterDate("2013-07-01", "2013-08-01"))
print("IC count:", ic.size().getInfo())

IC count: 31


In [4]:
lcz_img = ee.ImageCollection("RUB/RUBCLIM/LCZ/global_lcz_map/latest").first()
lcz = lcz_img.select("LCZ_Filter")
lst_scale_m = 1000
ring_outer_m= 12000
ring_inner_m= 3000
agg_func="median"
cell_crs="EPSG:3857"

start_date="2013-07-01"
end_date="2013-08-01"
unit="cell", # modify unit to 'cell'
cell_scale_m=500
lst_band="LST_Night_1km"
qc_band="QC_Night"

BUILT_MIN, BUILT_MAX = 1, 10
WATER_CODE = 17

is_built = lcz.gte(BUILT_MIN).And(lcz.lte(BUILT_MAX))
is_water = lcz.eq(WATER_CODE)
is_natural = is_built.Not().And(is_water.Not())

urban_region = city_geom
outer = city_geom.buffer(ring_outer_m)
inner = city_geom.buffer(ring_inner_m)
rural_region = outer.difference(inner)

urban_mask = is_water.Not().clip(urban_region)
rural_mask = is_natural.clip(rural_region)

In [5]:
grid_fc = uhi_pipeline.make_grid_fc_2(urban_region, cell_size_m=cell_scale_m, crs=cell_crs)

img = (ee.ImageCollection("MODIS/061/MYD11A1")
       .filterBounds(urban_region)
       .filterDate(start_date, end_date)
       .select([lst_band, qc_band])
       .map(lambda i: uhi_pipeline.clean_lst(i, lst_band, qc_band))
       .first())

combined = ee.Reducer.mean().combine(ee.Reducer.count(), sharedInputs=True)
urb_cells = img.updateMask(urban_mask).reduceRegions(
    collection=grid_fc, reducer=combined, scale=lst_scale_m, tileScale=4
)


print("urb first props:", urb_cells.first().toDictionary().getInfo())


urb first props: {'cell_id': '-12472000_3962644', 'count': 773, 'mean': 26.105032850438686}


In [6]:
print("before map keys:", urb_cells.first().propertyNames().getInfo())

def probe(ft):
    ft = ee.Feature(ft)
    return ft.set({
        "probe_urb": ft.get("mean"),
        "probe_cnt": ft.get("count")
    })

probed = urb_cells.map(probe)
print("after map keys:", probed.first().propertyNames().getInfo())
print("after map vals:", probed.first().toDictionary().getInfo())


before map keys: ['mean', 'count', 'system:index', 'cell_id']
after map keys: ['probe_cnt', 'probe_urb', 'mean', 'count', 'system:index', 'cell_id']
after map vals: {'cell_id': '-12472000_3962644', 'count': 773, 'mean': 26.105032850438686, 'probe_cnt': 773, 'probe_urb': 26.105032850438686}


In [7]:
img = ic.first()
urb = img.updateMask(urban_mask)
urb_cells = urb.reduceRegions(collection=grid_fc, reducer=ee.Reducer.count(), scale=lst_scale_m)
print("urb_cells size:", urb_cells.size().getInfo())

urb_cells size: 1


In [8]:
fc_raw = uhi_pipeline.make_daily_table_cells(
    start_date, end_date,
    urban_region, rural_region,
    urban_mask, rural_mask,
    lst_band, qc_band,
    agg_func,
    lst_scale_m=lst_scale_m,
    cell_scale_m=cell_scale_m,
    crs=cell_crs
)

print("raw fc size:", fc_raw.size().getInfo())
print("raw first feature:", fc_raw.toDictionary().getInfo())

raw fc size: 31
raw first feature: {}


In [9]:
fc_raw.first().toDictionary().getInfo()

{'LST_urb_cell': 26.086615598885825,
 'cell_id': '-12472000_3962644',
 'cell_n': 773,
 'date': '2013-07-01',
 'rural_n': 0}

In [10]:
print("raw with rural_n>=1:", fc_raw.filter(ee.Filter.gte("rural_n", 1)).size().getInfo())
print("raw with LST_rur not null:", fc_raw.filter(ee.Filter.notNull(["LST_rur"])).size().getInfo())

raw with rural_n>=1: 0
raw with LST_rur not null: 0


In [16]:
# 真正的 rural 检查：clean MODIS 在 rural_region 是否有值
img = (ee.ImageCollection("MODIS/061/MYD11A1")
       .filterBounds(urban_region)
       .filterDate(start_date, end_date)
       .select([lst_band, qc_band])
       .map(lambda i: uhi_pipeline.clean_lst(i, lst_band, qc_band))
       .first())

combined = ee.Reducer.mean().combine(ee.Reducer.count(), sharedInputs=True)

# A) 不加 LCZ rural_mask，只看 clean MODIS 在 rural_region
rur_nomask_stats = img.reduceRegion(
    reducer=combined,
    geometry=rural_region,
    scale=lst_scale_m,
    maxPixels=1e13
)
print("A) clean MODIS only in rural_region:", rur_nomask_stats.getInfo())

# B) 加上 LCZ rural_mask（这条才是你主流程真实在用的）
rur_masked = img.updateMask(rural_mask)
rur_masked_stats = rur_masked.reduceRegion(
    reducer=combined,
    geometry=rural_region,
    scale=lst_scale_m,
    maxPixels=1e13
)
print("B) clean MODIS + rural_mask in rural_region:", rur_masked_stats.getInfo())

A) clean MODIS only in rural_region: {'LST_Night_1km_count': 1686, 'LST_Night_1km_mean': 26.824349905041593}
B) clean MODIS + rural_mask in rural_region: {'LST_Night_1km_count': 1609, 'LST_Night_1km_mean': 26.753854532697357}


In [17]:
rur_stats = rur_masked_stats
# 假设 rur_stats 就是你刚算出来的 reduceRegion 字典
print(rur_stats.getInfo())  # 先看真实 key 名

lst_rur = ee.Algorithms.If(
    rur_stats.contains(f"{lst_band}_{agg_func}"),
    rur_stats.get(f"{lst_band}_{agg_func}"),
    rur_stats.get(agg_func)  # fallback
)

rural_n = ee.Algorithms.If(
    rur_stats.contains(f"{lst_band}_count"),
    rur_stats.get(f"{lst_band}_count"),
    rur_stats.get("count")   # fallback
)

{'LST_Night_1km_count': 1609, 'LST_Night_1km_mean': 26.753854532697357}


In [11]:
tmp = ee.FeatureCollection(fc_raw)
tmp = tmp.filter(ee.Filter.notNull(["date", "cell_id", "LST_urb_cell", "cell_n", "LST_rur", "rural_n"]))
tmp = tmp.filter(ee.Filter.gte("cell_n", 0))
tmp = tmp.filter(ee.Filter.gte("rural_n", 0))
print("after run_city-style filters:", tmp.size().getInfo())

after run_city-style filters: 0


In [12]:
# Example: Phoenix
importlib.reload(uhi_pipeline)
df = uhi_pipeline.run_city(
    ua_fc=ua_fc,
    ua_contains="Phoenix",
    start_date="2013-07-01",
    end_date="2013-08-01",
    unit="cell", # modify unit to 'cell'
    cell_scale_m=500,
    min_cell_pixels=0,
    min_rural_pixels=0,
    agg_func="median",
    lst_band="LST_Night_1km",
    qc_band="QC_Night",
)

print(df.shape)
print(df.head())
# print(df["date"].min(), df["date"].max())
# print(df[["cell_n","rural_n"]].describe())

fc_list length: 1
first fc size: 0
No features returned after filtering. Retrying with relaxed pixel thresholds: urban>=1, rural>=1, cell>=1.
No features returned from Earth Engine after filtering. Returning empty DataFrame.
(0, 0)
Empty DataFrame
Columns: []
Index: []


Displaying the current content of `uhi_pipeline.py` to verify the applied changes.

In [13]:
with open('/content/uhi_pipeline.py', 'r') as f:
    print(f.read())

"""
Reusable SUHI data pipeline (GEE + MODIS LST)
Optimized Version: Uses Combined Reducers to reduce server load.
"""

import ee
import geemap
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

# ------------------------------------------------------------------
# 0. Earth Engine init
# ------------------------------------------------------------------
def init_ee():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# ------------------------------------------------------------------
# 1. Helper: monthly ranges
# ------------------------------------------------------------------
def month_starts(start_date, end_date):
    s = datetime.strptime(start_date, "%Y-%m-%d").replace(day=1)
    e = datetime.strptime(end_date, "%Y-%m-%d")
    cur = s
    while cur < e:
        nxt = cur + relativedelta(months=1)
        yield cur.strftime("%Y-%m-%d"), nxt.strftime("%Y-%m-%